In [10]:
import os
import numpy as np
from scipy.ndimage import imread
import cv2
import sklearn.utils

DATA_PATH = './data/train/'
TEST_PERCENT = 0.4;
SELECT_SUBSET_PERCENT = 0.15

#cat and dog images are od variable size
RESIZE_WIDTH = 32
RESIZE_HEIGHT = 32
EPOCHS = 5

In [15]:
X = []
Y = []

files = os.listdir(DATA_PATH)
# Shuffle to select equal number of cat and god images
shuffled_files = sklearn.utils.shuffle(files)
select_count = int(len(shuffled_files) * SELECT_SUBSET_PERCENT)

print('Going to load %i files' % select_count)
subset_files_select = shuffled_files[:select_count]
DISPLAY_COUNT = 1000

for i, input_file in enumerate(subset_files_select):
    if i % DISPLAY_COUNT == 0 and i != 0:
        print('Have loaded %i samples' %i)
        
    img = imread(DATA_PATH + input_file)
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    X.append(img)
    
    # labelling the data
    if 'cat' == input_file.split('.')[0]:
        Y.append(0.0)
    else:
        Y.append(1.0)
    
X = np.array(X)
Y = np.array(Y)

test_size = int(len(X) * 0.05)

test_X = X[:test_size]
test_Y = Y[:test_size]
train_X = X[test_size:]
train_Y = Y[test_size:]

print('Train set has dimensionality %s' % str(train_X.shape))
print('Test set has dimensionality %s' % str(test_X.shape))

# Apply some normalization here.
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X /= 255
test_X /= 255
    


Going to load 3538 files
Have loaded 1000 samples
Have loaded 2000 samples
Have loaded 3000 samples
Train set has dimensionality (3362, 32, 32, 3)
Test set has dimensionality (176, 32, 32, 3)


In [ ]:
# Import necessary layers.
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.constraints import max_norm


model = Sequential()

# Define the network
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', input_shape=(RESIZE_WIDTH, RESIZE_HEIGHT, 3)))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512, kernel_initializer='glorot_normal'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='glorot_normal'))
model.add(Activation('sigmoid'))


# Defining loss and optimizer
optimizer = RMSprop(lr=1e-4)
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])


In [ ]:
batch_size = 64

# test the network:
model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 2689 samples, validate on 673 samples
Epoch 1/5
2689/2689 [==============================] - 28s - loss: 0.6877 - acc: 0.5400 - val_loss: 0.6919 - val_acc: 0.5364
Epoch 2/5
2689/2689 [==============================] - 24s - loss: 0.6820 - acc: 0.5444 - val_loss: 0.6878 - val_acc: 0.5171
Epoch 3/5
2689/2689 [==============================] - 23s - loss: 0.6771 - acc: 0.5608 - val_loss: 0.7285 - val_acc: 0.5364
Epoch 4/5
2689/2689 [==============================] - 25s - loss: 0.6740 - acc: 0.5664 - val_loss: 0.6743 - val_acc: 0.5899
Epoch 5/5
2240/2689 [=======================>......] - ETA: 3s - loss: 0.6586 - acc: 0.6094

In [ ]:
loss, acc = model.evaluate(test_X, test_Y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))